In [1]:
import sys

from pathlib import Path
import numpy as np
import pandas as pd
import scipy.signal
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import copy
import pickle


import warnings
warnings.simplefilter("ignore")

In [2]:
class CFG:
    exp_num = 1
    n_folds = 10
    folds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    seed = 777
    local = True


In [3]:
DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
OUTPUT_DIR = Path('./output/')
OOF_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking")
SUB_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking")
PICKLE_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_stacking_pickle")

In [7]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [8]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")

In [9]:
with open(PICKLE_DIR / "oof_ver1101_1.pkl", mode="rb") as f:
    oofs = pickle.load(f)

In [12]:
oofs.columns = [f"pred_{i}" for i in range(len(oofs.columns))]
oofs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18
0,5.843223,6.287325,5.798063,5.828472,5.783465,6.013763,6.251595,5.823515,6.477747,5.843684,7.629176,5.816144,6.452471,5.834950,6.206788,6.402203,5.788936,5.816258,5.750221
1,5.873457,5.989612,5.848358,5.880560,5.869486,6.103593,5.920428,5.898210,6.534627,5.854125,6.086765,5.794350,5.884715,5.841093,5.789935,5.939039,5.865545,5.818817,5.700664
2,7.892688,8.725241,7.790454,8.157262,7.865620,8.852818,8.440607,7.930587,8.846078,7.914377,8.957220,7.781632,8.618551,7.889086,8.631878,8.335553,7.870878,7.932899,8.225599
3,11.861191,12.752718,11.766545,12.129442,11.779996,12.312957,12.190743,12.042971,12.801433,11.954165,12.609935,11.814193,12.376060,11.697797,12.434288,12.549238,11.836457,11.926755,12.087312
4,12.331974,12.974612,12.218771,12.576923,12.318006,12.656638,12.622059,12.474959,13.259793,12.393296,12.761421,12.382209,12.714886,12.329643,12.796815,12.954475,12.394281,12.622350,12.770415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,3.878536,10.053699,27.399870,33.113632,23.812657,9.911885,9.750838,50.085614,7.869389,3.867466,14.699379,3.898078,8.355096,3.899063,9.972795,9.540561,29.231508,3.909052,40.662281
6035996,3.888600,10.087551,29.374889,33.605228,23.587521,9.934764,9.728185,49.767257,11.205536,3.872056,14.747525,3.905671,8.279023,3.901155,9.939343,9.791974,29.102717,3.898015,42.027161
6035997,3.815409,9.693629,27.324729,33.979126,22.652757,9.812245,9.473552,32.764555,11.790893,3.791186,17.724089,3.847541,8.294396,3.842515,9.892908,9.687879,28.371311,3.817773,42.148769
6035998,4.180053,10.223997,31.433994,31.554214,26.580203,8.975171,10.027510,27.734379,10.896358,4.091510,7.662900,4.149836,11.226018,4.139171,11.345486,9.264953,30.142678,4.119548,41.050766


In [13]:
with open(PICKLE_DIR / "pred_ver1101_1.pkl", mode="rb") as f:
    subs = pickle.load(f)

In [14]:
subs.columns = [f"pred_{i}" for i in range(len(subs.columns))]
subs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18
0,6.237001,6.190098,6.286294,6.258121,6.315998,6.157633,6.153747,6.301814,6.130114,6.295876,6.113052,6.245053,6.208011,6.291997,6.191348,5.868285,6.307113,6.306786,6.246800
1,5.938651,6.075787,5.926586,5.953184,5.938564,6.029568,6.061356,5.943007,6.060609,5.965106,5.969356,5.935810,6.033892,5.990556,6.055338,5.722659,5.943791,5.968692,5.954641
2,7.218726,7.236039,7.109683,7.180656,7.115508,7.136393,7.312749,7.136607,7.189013,7.117581,7.113848,7.179698,7.160792,7.142857,7.202180,6.966420,7.128859,7.112571,7.190442
3,7.752637,7.880373,7.632365,7.789594,7.607679,7.800501,7.841540,7.651355,7.909873,7.631863,7.706476,7.734634,7.800667,7.654799,7.776278,7.603629,7.615316,7.638148,7.785542
4,9.262370,9.373093,9.117007,9.275614,9.145709,9.164462,9.336922,9.160076,9.389410,9.132385,9.256947,9.260639,9.270784,9.161454,9.304310,9.119343,9.128102,9.155414,9.280422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,6.187890,14.774168,13.814573,35.090607,15.863170,18.102705,16.602873,18.055397,11.708288,6.150437,12.287502,6.208994,14.476089,6.163520,16.766878,15.343576,15.152164,6.201168,39.418259
4023996,6.290313,14.924815,14.047708,35.653534,16.148630,18.367050,17.279026,20.852467,11.723017,6.322860,12.271587,6.269710,14.706350,6.320943,16.987122,14.469177,15.314502,6.285121,40.991684
4023997,6.397727,14.827735,13.941481,36.066963,16.197741,17.861053,17.451265,22.164679,12.104719,6.358964,12.555499,6.435824,14.665152,6.379314,16.780479,15.494080,15.419031,6.437337,41.936821
4023998,6.175601,16.304645,14.051582,34.567181,16.050034,16.602900,17.901031,23.106366,12.155079,6.156576,13.272074,6.193621,15.665402,6.157371,17.971500,16.013962,15.357834,6.179539,39.998093


In [15]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'R', 'C']

    return input_df[colum]

In [16]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [17]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df['u_in_sqrt'] = output_df['u_in'].apply(lambda x: np.sqrt(x))
    output_df['u_in_sqrt_cumsum'] = output_df.groupby('breath_id')['u_in_sqrt'].cumsum()
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [18]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [19]:
def get_half_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['tmp'] = output_df['u_out']*(-1)+1 # inversion of u_out
    output_df['u_in_half'] = output_df['tmp'] * output_df['u_in']
    
#     u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
#     train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
#     u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
#     train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
    u_in_half_mean_dict = output_df.groupby('breath_id')['u_in_half'].mean().to_dict()
    output_df['u_in_half_mean'] = output_df['breath_id'].map(u_in_half_mean_dict)
#     u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
#     train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

    del output_df['u_in_half'], output_df['tmp']
    return output_df.iloc[:, c_num:]

In [20]:
def lowpass_filter(series, b, a):
    return signal.filtfilt(b, a, series)    

In [21]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    fp = 5 # 通過域端周波数[Hz]
    fs = 10 # 阻止域端周波数[Hz]
    gpass = 3 # 通過域端最大損失[dB]
    gstop = 40 # 阻止域端最小損失[dB]
    samplerate = 100

    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter, b=b, a=a)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    df_agg_feature['u_in_filter'] = df_agg_feature['u_in_filter'].astype(float)
        
    df_agg_feature['u_in_filter_cumsum'] = df_agg_feature.groupby('breath_id')['u_in_filter'].cumsum()

    return df_agg_feature.iloc[:, 1:]

In [22]:
def get_vib_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['u_out_diff'] = output_df['u_out'].diff()
    output_df['u_out_diff'].fillna(0, inplace=True)
    output_df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = output_df[output_df['u_out_diff']==1]
    
    first_df = output_df.loc[0::80,:]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    output_df['u_in_diff'] = output_df['u_in'].diff()
    output_df['diff_sign'] = np.sign(output_df['u_in_diff'])
    output_df['sign_diff'] = output_df['diff_sign'].diff()
    output_df['tmp'] = output_df['id'].map(first_0_dict) # put 0, the 80row cycle
    output_df.iloc[0::80, output_df.columns.get_loc('sign_diff')] = output_df.iloc[0::80, output_df.columns.get_loc('tmp')]

    # Count the number of inversions, so take the absolute value and sum
    output_df['sign_diff'] = abs(output_df['sign_diff']) 
    sign_diff_dict = output_df.groupby('breath_id')['sign_diff'].sum().to_dict()
    output_df['diff_vib'] = output_df['breath_id'].map(sign_diff_dict)
    
    return output_df['sign_diff']

In [23]:
def add_time_features(out_df, input_df, dataType = 'train'):

    USE_LAG = [-2, -1, 1, 2, 3, 4]
    lag_map = {-2: 1, -1: 2, 1: 3, 2: 4, 3: 5, 4: 6}

    out_df['breath_id'] = input_df['breath_id']
    
    for lag in USE_LAG:
        out_df[f'breath_id_lag{lag_map[lag]}']=out_df['breath_id'].shift(lag).fillna(0)
        out_df[f'breath_id_lag{lag_map[lag]}same']=np.select([out_df[f'breath_id_lag{lag_map[lag]}']==out_df['breath_id']], [1], 0)

        # u_in_filter
        out_df[f'u_in_filter_lag_{lag_map[lag]}'] = out_df['u_in_filter'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_filter_diff_{lag_map[lag]}'] = out_df['u_in_filter'] - out_df[f'u_in_filter_lag_{lag_map[lag]}']
        # u_in_sqrt
        out_df[f'u_in_sqrt_lag_{lag_map[lag]}'] = out_df['u_in_sqrt'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_sqrt_diff_{lag_map[lag]}'] = out_df['u_in_sqrt'] - out_df[f'u_in_sqrt_lag_{lag_map[lag]}']

        # u_in 
        out_df[f'u_in_lag_{lag_map[lag]}'] = out_df['u_in'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_diff_{lag_map[lag]}'] = out_df['u_in'] - out_df[f'u_in_lag_{lag_map[lag]}']
        # u_out
        out_df[f'u_out_lag_{lag_map[lag]}'] = out_df['u_out'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']

        # breath_time
    out_df[f'time_step_lag_{1}'] = out_df['time_step'].shift(1).fillna(0) * out_df[f'breath_id_lag{1}same']
    out_df[f'time_step_diff_{1}'] = out_df['time_step'] - out_df[f'time_step_lag_{1}']
        
    drop_columns = ['breath_id', 'time_step_lag_1']
    drop_columns += [f'breath_id_lag{lag_map[i]}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{lag_map[i]}same' for i in USE_LAG]
    out_df = out_df.drop(drop_columns, axis=1)
    out_df = out_df.fillna(0)
    
    return out_df

In [24]:
def get_oof_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    for i in range(len(pred_cols)):
        output_df[f"pred_{i}"] = 0.
        output_df.loc[oof["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    

    
    return output_df['sign_diff']

In [25]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
#         get_simple_calc_features,
#         get_agg_features,
#         get_vib_features,
#         get_half_features,
        get_category_features,
#         get_filter_features,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
#     out_df = add_time_features(out_df, input_df)
    out_df_cols = sorted(list(out_df))
    out_df = out_df[out_df_cols]
    
    return out_df

In [45]:
train_df = to_feature(train, dataType = 'train')
test_df = to_feature(test, dataType = 'test')

  0%|          | 0/2 [00:00<?, ?it/s]

get_raw_features  0.029[s]


  0%|          | 0/2 [00:00<?, ?it/s]

get_category_features  2.399[s]
get_raw_features  0.015[s]


100%|██████████| 2/2 [00:01<00:00,  1.31it/s]

get_category_features  1.487[s]


In [46]:
pred_cols = [f"pred_{i}" for i in range(len(oofs.columns))]

In [47]:
def add_oof_features(input_df, dataType = 'train'):
    
    if dataType == 'train':
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[:, f"pred_{i}"] = oofs[f"pred_{i}"].values
        input_df['breath_id'] = train['breath_id']
        input_df = train_df.loc[train["u_out"] == 0].reset_index(drop=True)
    else:
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[:, f"pred_{i}"] = subs[f"pred_{i}"].values
        input_df['breath_id'] = test['breath_id']
        input_df = input_df.loc[test["u_out"] == 0].reset_index(drop=True)
      
    # v2
    input_df["pred_mean"] = np.mean(input_df[pred_cols].values, axis=1)
    input_df["pred_median"] = np.median(input_df[pred_cols].values, axis=1)

    input_df["pred_std"] = input_df[pred_cols].std(axis=1)
    input_df["pred_max"] = input_df[pred_cols].values.max(axis=1)
    input_df["pred_min"] = input_df[pred_cols].values.min(axis=1)
    input_df["pred_max-min"] = input_df["pred_max"] - input_df["pred_min"]
    input_df["pred_max-median"] = input_df["pred_max"] - input_df["pred_median"]
    input_df["pred_max-mean"] = input_df["pred_max"] - input_df["pred_mean"]
    input_df["pred_median-min"] = input_df["pred_median"] - input_df["pred_min"]
    input_df["pred_mean-min"] = input_df["pred_mean"] - input_df["pred_min"]
    input_df["pred_mean-median"] = input_df["pred_mean"] - input_df["pred_median"]
    input_df["pred_kurt"] = input_df[pred_cols].kurt(axis=1)
    for col_ in pred_cols:
        input_df[f"{col_}_past_1"] = input_df.groupby("breath_id")[f"{col_}"].shift(1)
        input_df[f"{col_}_past_2"] = input_df.groupby("breath_id")[f"{col_}"].shift(2)
        input_df[f"{col_}_past_3"] = input_df.groupby("breath_id")[f"{col_}"].shift(3)
        input_df[f"{col_}_past_4"] = input_df.groupby("breath_id")[f"{col_}"].shift(4)

        input_df[f"{col_}_diff_1"] = input_df[f"{col_}"] - input_df[f"{col_}_past_1"]
        input_df[f"{col_}_diff_2"] = input_df[f"{col_}"] - input_df[f"{col_}_past_2"]
        input_df[f"{col_}_diff_3"] = input_df[f"{col_}"] - input_df[f"{col_}_past_3"]
        input_df[f"{col_}_diff_4"] = input_df[f"{col_}"] - input_df[f"{col_}_past_4"]

    input_df["u_in_past_1"] = input_df.groupby("breath_id")["u_in"].shift(1)
    input_df["u_in_past_2"] = input_df.groupby("breath_id")["u_in"].shift(2)
    input_df["u_in_past_3"] = input_df.groupby("breath_id")["u_in"].shift(3)
    input_df["u_in_past_4"] = input_df.groupby("breath_id")["u_in"].shift(4)

    input_df["u_in_diff_1"] = input_df["u_in"] - input_df["u_in_past_1"]
    input_df["u_in_diff_2"] = input_df["u_in"] - input_df["u_in_past_2"]
    input_df["u_in_diff_3"] = input_df["u_in"] - input_df["u_in_past_3"]
    input_df["u_in_diff_4"] = input_df["u_in"] - input_df["u_in_past_4"]

    input_df["u_in_cumsum"] = input_df.groupby("breath_id")["u_in"].cumsum()

    del input_df['breath_id']
    
    return input_df

In [48]:
train_df = add_oof_features(train_df, dataType = 'train')
test_df = add_oof_features(test_df, dataType = 'test')

In [49]:
display(train_df), display(test_df)

,C,R,R_C,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_18_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,50,20,5,0.000000,0.083334,5.843223,6.287325,5.798063,5.828472,5.783465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083334
1,50,20,5,0.033652,18.383041,5.873457,5.989612,5.848358,5.880560,5.869486,...,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN,18.466375
2,50,20,5,0.067514,22.509278,7.892688,8.725241,7.790454,8.157262,7.865620,...,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN,40.975653
3,50,20,5,0.101542,22.808822,11.861191,12.752718,11.766545,12.129442,11.779996,...,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN,63.784476
4,50,20,5,0.135756,25.355850,12.331974,12.974612,12.218771,12.576923,12.318006,...,7.020194,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516,89.140326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,6,0.834147,1.869367,29.492047,29.437429,29.559149,29.562796,29.377239,...,1.145603,2.650333,2.438287,3.783043,3.209590,-0.780965,-0.568920,-1.913676,-1.340223,238.890288
2290964,10,50,6,0.867574,2.154414,29.098768,29.094450,29.225697,29.233667,29.051178,...,1.379343,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629,241.044703
2290965,10,50,6,0.900917,1.304434,29.777598,29.816580,30.003416,29.977200,29.847507,...,0.843889,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853,242.349137
2290966,10,50,6,0.934309,1.733830,29.344897,29.375248,29.511444,29.443771,29.373464,...,0.568394,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503,244.082966


,C,R,R_C,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_18_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,20,5,1,0.000000,0.000000,6.237001,6.190098,6.286294,6.258121,6.315998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,20,5,1,0.031904,7.515046,5.938651,6.075787,5.926586,5.953184,5.938564,...,NaN,0.000000,NaN,NaN,NaN,7.515046,NaN,NaN,NaN,7.515046
2,20,5,1,0.063827,14.651675,7.218726,7.236039,7.109683,7.180656,7.115508,...,NaN,7.515046,0.000000,NaN,NaN,7.136630,14.651675,NaN,NaN,22.166721
3,20,5,1,0.095751,21.230610,7.752637,7.880373,7.632365,7.789594,7.607679,...,NaN,14.651675,7.515046,0.000000,NaN,6.578935,13.715564,21.230610,NaN,43.397331
4,20,5,1,0.127644,26.320956,9.262370,9.373093,9.117007,9.275614,9.145709,...,3.033622,21.230610,14.651675,7.515046,0.0,5.090346,11.669281,18.805911,26.320956,69.718287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527560,10,20,3,0.842145,0.000000,10.120168,10.109750,10.096148,10.103710,10.101749,...,0.006554,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,68.839645
1527561,10,20,3,0.875648,0.000000,10.102587,10.094096,10.085585,10.086064,10.093045,...,-0.049212,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,68.839645
1527562,10,20,3,0.909185,0.121375,9.986692,9.984508,9.983848,9.993743,9.985178,...,-0.147991,0.000000,0.000000,0.000000,0.0,0.121375,0.121375,0.121375,0.121375,68.961019
1527563,10,20,3,0.943148,0.000000,10.080103,10.081043,10.058499,10.095322,10.078216,...,-0.024134,0.121375,0.000000,0.000000,0.0,-0.121375,0.000000,0.000000,0.000000,68.961019


(None, None)

In [50]:
train_category_col = ['R_C']
train_value_col = [i for i in train_df.columns.to_list() if i not in train_category_col]

In [51]:
norm_features = train_value_col
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
scaler = RobustScaler()
scaler.fit(train_df[norm_features])
train_df[norm_features] = scaler.transform(train_df[norm_features].values)
test_df[norm_features] = scaler.transform(test_df[norm_features].values)

In [52]:
y = train.loc[train["u_out"] == 0]['pressure'].reset_index(drop=True)

In [53]:
train_df = pd.concat([train_df, train.loc[train["u_out"] == 0, ['id', 'breath_id', 'pressure']].reset_index(drop=True)], axis=1)
test_df = pd.concat([test_df, test.loc[test["u_out"] == 0,['id', 'breath_id']].reset_index(drop=True)], axis=1)

In [54]:
oof_prediction = np.zeros(len(train_df))
test_preds_lst = []
input_dim = len(train_value_col)
train_df['pred'] = 0
train_gby = train_df.groupby('breath_id')['R_C'].agg('first').reset_index()
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
models = []


fold_df = pd.DataFrame()
fold_df["id"] = train["id"]
fold_df["fold"] = -1

for fold, (_, valid_idx) in enumerate(skf.split(train_gby, train_gby['R_C'])):        
    valid_b_ids = train_gby.iloc[valid_idx]['breath_id'].values
    valid_df_idx = train_df[train_df['breath_id'].isin(valid_b_ids)].index.to_list()
    fold_df.loc[valid_df_idx, 'fold'] = fold

for i, fold in enumerate(range(CFG.n_folds)):
    if i not in CFG.folds:
        continue
    print(f'Fold-{fold}')
    
    train_idx = fold_df[fold_df["fold"] != fold].index
    valid_idx = fold_df[fold_df["fold"] == fold].index
    
    trn_df = train_df.loc[fold_df["fold"] != fold, train_value_col].reset_index(drop=True)
    val_df = train_df.loc[fold_df["fold"] == fold, train_value_col].reset_index(drop=True)
    trn_y = train_df.loc[fold_df["fold"] != fold, 'pressure'].reset_index(drop=True)
    val_y = train_df.loc[fold_df["fold"] == fold, 'pressure'].reset_index(drop=True)
    
    model = RidgeCV()
    model.fit(trn_df, trn_y)
    
    models.append(model)
    
    oof_prediction[valid_idx] = model.predict(val_df[train_value_col])
    test_pred = model.predict(test_df[train_value_col])
    test_preds_lst.append(test_pred)
    score = np.abs(val_y.values - oof_prediction[valid_idx]).mean()
    print(f'fold = {fold}, score = {score}')

Fold-0
fold = 0, score = 0.12092234355612247
Fold-1
fold = 1, score = 0.12477349950009843
Fold-2
fold = 2, score = 0.11964469722241677
Fold-3
fold = 3, score = 0.12325219342520088
Fold-4
fold = 4, score = 0.122062330274114
Fold-5
fold = 5, score = 0.12267550310592569
Fold-6
fold = 6, score = 0.12165790111811928
Fold-7
fold = 7, score = 0.12410296231660915
Fold-8
fold = 8, score = 0.12246003474812395
Fold-9
fold = 9, score = 0.12229111945724261


In [55]:
CV = np.abs(y.values - oof_prediction).mean()
CV

0.12238424997312247

In [61]:
np.save(OUTPUT_DIR / f"stacking_oof_{CFG.exp_num}", oof_prediction)

In [58]:
unique_pressures = train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

In [59]:
oof = pd.DataFrame({'pred': oof_prediction})
oof_pp = oof['pred'].map(lambda x: unique_pressures[np.abs(unique_pressures-x).argmin()])
score = np.abs(y.values - oof_pp).mean()
print(score)

0.11979720423723357


In [60]:
sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

sub_df.loc[test['u_out']==0, 'pressure'] = np.stack(test_preds_lst).mean(0)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_mean_{CFG.exp_num}.csv", index=None)

sub_df.loc[test['u_out']==0, 'pressure'] = np.median(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_median_{CFG.exp_num}.csv", index=None)

# Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess


sub_df = pd.read_csv(OUTPUT_DIR / f"stacking_submission_mean_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_mean_pp_{CFG.exp_num}.csv", index=None)

sub_df = pd.read_csv(OUTPUT_DIR / f"stacking_submission_median_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_median_pp_{CFG.exp_num}.csv", index=None)